# Exercises of difficulty 4 in R

In [3]:
library(testthat)

# 1. Strings Mix

Given two strings s1 and s2, we want to visualize how different the two strings 
are. We will only take into account the lowercase letters (a to z). First let us
count the frequency of each lowercase letters in s1 and s2.

    s1 = "A aaaa bb c"
    s2 = "& aaa bbb c d"

    --> s1 has 4 'a', 2 'b', 1 'c'
    --> s2 has 3 'a', 3 'b', 1 'c', 1 'd'
    
So the maximum for 'a' in s1 and s2 is 4 from s1; the maximum for 'b' is 3 from s2.
In the following we will not consider letters when the maximum of their 
occurrences is less than or equal to 1.

We can resume the differences between s1 and s2 in the following string: 

    "1:aaaa/2:bbb" 
where 1 in 1:aaaa stands for string s1 and aaaa because the maximum for a is 4. 
In the same manner 2:bbb stands for string s2 and bbb because the maximum for b is 3.

The task is to produce a string in which each lowercase letters of s1 or s2 
appears as many times as its maximum if this maximum is strictly greater than 1; 
these letters will be prefixed by the number of the string where they appear with
their maximum value and :. If the maximum is in s1 as well as in s2 the prefix is E:.

In the result, substrings (a substring is for example 2:nnnnn or 1:hhh; it contains
the prefix) will be in decreasing order of their length and when they have the same
length sorted in ascending lexicographic order (letters and digits - more precisely
sorted by codepoint); the different groups will be separated by '/'. 

Hopefully other examples can make this clearer.

### Examples

    s1 = "my&friend&Paul has heavy hats! &"
    s2 = "my friend John has many many friends &"
    mix(s1, s2) --> "2:nnnnn/1:aaaa/1:hhh/2:mmm/2:yyy/2:dd/2:ff/2:ii/2:rr/E:ee/E:ss"

    s1 = "mmmmm m nnnnn y&friend&Paul has heavy hats! &"
    s2 = "my frie n d Joh n has ma n y ma n y frie n ds n&"
    mix(s1, s2) --> "1:mmmmmm/E:nnnnnn/1:aaaa/1:hhh/2:yyy/2:dd/2:ff/2:ii/2:rr/E:ee/E:ss"

    s1="Are the kids at home? aaaaa fffff"
    s2="Yes they are here! aaaaa fffff"
    mix(s1, s2) --> "E:aaaaaa/2:eeeee/E:fffff/1:tt/2:rr/E:hh"

In [4]:
# Function to count the letters (a-z) in a string and return it as table
count_letters <- function(s) {
  table(factor(unlist(strsplit(gsub("[^a-z]", "", s, perl = TRUE), "")), 
               levels = letters))
}

mix <- function(s1, s2) {

  # Count letters of s1 and s2 
  cl1 <- count_letters(s1)
  cl2 <- count_letters(s2)
  
  # Get the results by comparing frequencies at each letter
  out <- ifelse(cl1 > cl2, paste0("1:", strrep(names(cl1), cl1)),
         ifelse(cl2 > cl1, paste0("2:", strrep(names(cl2), cl2)),
                paste0("E:", strrep(names(cl2), cl2))))
  
  # Only keep the entrances with at least 3 characters 
  out <- out[nchar(out) > 3]
  
  # Paste the values to a string and return it
  paste(out[order(-nchar(out), rank(substr(out,1, 1)))], collapse = "/")
}

In [5]:
testing <- function(s1, s2, expected) {
  actual <- mix(s1, s2)
  cat("actual   ", actual, "\n")
  cat("expected ", expected, "\n ------ \n")
  expect_equal(actual, expected)
}


test_that("tests", {
  testing("Are they here", "yes, they are here", 
          "2:eeeee/2:yy/E:hh/E:rr")
  testing("uuuuuu", "uuuuuu", "E:uuuuuu")
  testing("looping is fun but dangerous", "less dangerous than coding", 
          "1:ooo/1:uuu/2:sss/E:nnn/1:ii/2:aa/2:dd/2:ee/E:gg")
  testing(" In many languages", " there's a pair of functions", 
          "1:aaa/1:nnn/1:gg/2:ee/2:ff/2:ii/2:oo/2:rr/2:ss/2:tt")
  testing("Lords of the Fallen", "gamekult", "1:ee/1:ll/1:oo")
  testing("codewars", "codewars", "")
  testing("A generation must confront the looming ", 
          "codewarrs", "1:nnnnn/1:ooooo/1:tttt/1:eee/1:gg/1:ii/1:mm/E:rr")

})

actual    2:eeeee/2:yy/E:hh/E:rr 
expected  2:eeeee/2:yy/E:hh/E:rr 
 ------ 
actual    E:uuuuuu 
expected  E:uuuuuu 
 ------ 
actual    1:ooo/1:uuu/2:sss/E:nnn/1:ii/2:aa/2:dd/2:ee/E:gg 
expected  1:ooo/1:uuu/2:sss/E:nnn/1:ii/2:aa/2:dd/2:ee/E:gg 
 ------ 
actual    1:aaa/1:nnn/1:gg/2:ee/2:ff/2:ii/2:oo/2:rr/2:ss/2:tt 
expected  1:aaa/1:nnn/1:gg/2:ee/2:ff/2:ii/2:oo/2:rr/2:ss/2:tt 
 ------ 
actual    1:ee/1:ll/1:oo 
expected  1:ee/1:ll/1:oo 
 ------ 
actual     
expected   
 ------ 
actual    1:nnnnn/1:ooooo/1:tttt/1:eee/1:gg/1:ii/1:mm/E:rr 
expected  1:nnnnn/1:ooooo/1:tttt/1:eee/1:gg/1:ii/1:mm/E:rr 
 ------ 


# 2. Simple Fun #27: Rectangle Rotation

A rectangle with sides equal to even integers a and b is drawn on the Cartesian 
plane. Its center (the intersection point of its diagonals) coincides with the 
point (0, 0), but the sides of the rectangle are not parallel to the axes; 
instead, they are forming 45 degree angles with the axes.

How many points with integer coordinates are located inside the given rectangle 
(including on its sides)?

### Example
For a = 6 and b = 4, the output should be 23

The following picture illustrates the example, and the 23 points are marked green.
(see https://www.codewars.com/kata/5886e082a836a691340000c3/train/r)

### Input/Output
    > integer a: A positive even integer - 2 ≤ a ≤ 10000.
    > integer b: A positive even integer - 2 ≤ a ≤ 10000.

    > output: The number of inner points with integer coordinates.
    

In [10]:
rectangle_rotation <- function(a, b) {
    
    # Get the amount of points covered 
    #  -> Each square is 1x1 & hence it's diagonal is sqrt(2), due to [a^2+b^2=c^2]
    #  -> Get the amount of squares covered in two ways:
    #    (1) 
    
    #  -> 
    
    # Amount of points covered in the direction of length of a // b
    #  - Get amounts covered by length + add 1 due to center-point (0/0)
  
    p_a_l <- floor((a / sqrt(2)) / 2) * 2 + 1 
    p_b_l <- floor((b / sqrt(2)) / 2) * 2 + 1 
    
    p_a_s <- 2 * ceiling(((a/2) - sqrt(2)/2) / sqrt(2))
    p_b_s <- 2 * ceiling(((b/2) - sqrt(2)/2) / sqrt(2))

    return(p_b_s * p_a_s + p_b_l * p_a_l)
}

In [241]:
a = 6
b = 4

  #  -> div by sqrt(2), as every 'sqrt(2)' there is a new point (1^2 + 1^2 = diagonale quadrat)

floor(a / sqrt(2)) + 1
floor((a / sqrt(2)) / 2) * 2 + 1 

floor((b / sqrt(2)))

[1] 5

[1] 5

[1] 2

In [11]:
test_that("basic tests", {
  expect_equal(rectangle_rotation(6, 4), 23)
  expect_equal(rectangle_rotation(30, 2), 65)
  expect_equal(rectangle_rotation(8, 6), 49)
  expect_equal(rectangle_rotation(16, 20), 333)
})

# 12. Twice linear
Consider a sequence u where u is defined as follows:

The number u(0) = 1 is the first one in u.
For each x in u, then y = 2 * x + 1 and z = 3 * x + 1 must be in u too.
There are no other numbers in u.

#### Example: 
    
    u = [1, 3, 4, 7, 9, 10, 13, 15, 19, 21, 22, 27, ...]
    - 1 gives 3 and 4
    - 3 gives 7 and 10
    - 4 gives 9 and 13
    - 7 gives 15 and 22 
    - and so on...

#### Task:
Given parameter n the function dbl_linear (or dblLinear...) returns the element u(n) of the ordered (with <) sequence u (so, there are no duplicates).

#### Example:
dbl_linear(10) should return 22

#### Note:
Focus attention on efficiency

In [225]:
# Write Function
dblLinear <- function (n) {
    
    # Define vector 'u' w/ the start-value '1' 
    # & the index counter variable
    u   <- c(1) 

    # Calculate the new values in 'u' until it's length is >= n * 10
    while (length(u) < n * 10) {

        # Calc the next two positions & add them to 'u'
        u <- sort(unique(c(u, 2 * u + 1, 3 * u + 1)))
    }
    
    u[n + 1]
}

In [226]:
# Test it
testing <- function(n, expected) {
  actual <- dblLinear(n)
  expect_equal(actual, expected)
}
test_that("tests", {
  testing(10, 22)
  testing(20, 57)
})